In [1]:
[1,1]

2-element Array{Int64,1}:
 1
 1

In [1]:
"""
DDP problem settings.
# Arguments
- `T :: Int64`: the time.
- `N :: Int64`: the discretization of time.
- `x0 
-
...
"""
struct DDP
    T :: Int64
    N :: Int64
    x0 :: Array{Float64, 1}
    p :: Array{Float64, 1}
    
    DDP(T, N, x0, p) = 
        size(x0) != size(p) ? 
        error("x0 and p must have the same size") :
        new(T, N, x0, p)
    
end

DDP

In [6]:
using ReverseDiff

In [9]:
function F(x, u, h = 0.01, d = 4) 
    x, y, theta, v = x
    w, a = u
    
    f = h*v
    b = f*cos(w) + d - sqrt(d^2 - f^2 * sin(w)^2)

    x = x + b*cos(theta)
    y = y + b*sin(theta)
    theta = theta + asin(sin(w) * f/d)
    v = v + h*a

    return [x, y, theta, v]
    
end

F (generic function with 3 methods)

In [9]:
a = ReverseDiff.jacobian(F, ([1, 1, 3/2 * pi, 0], [1, 1]))

([1.0 0.0 0.0 -9.92519e-19; 0.0 1.0 0.0 -0.00540302; 0.0 0.0 1.0 0.00210368; 0.0 0.0 0.0 1.0], [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.01])

In [11]:
typeof(a)

Tuple{Array{Float64,2},Array{Float64,2}}

In [12]:
a[1]

4×4 Array{Float64,2}:
 1.0  0.0  0.0  -9.92519e-19
 0.0  1.0  0.0  -0.00540302 
 0.0  0.0  1.0   0.00210368 
 0.0  0.0  0.0   1.0        

In [13]:
a[2]

4×2 Array{Float64,2}:
 0.0  0.0 
 0.0  0.0 
 0.0  0.0 
 0.0  0.01

In [14]:
using ForwardDiff

In [15]:
function F(xu::Vector, h = 0.01, d = 4) 
    x, y, theta, v, w, a = xu
#     w, a = u
    
    f = h*v
    b = f*cos(w) + d - sqrt(d^2 - f^2 * sin(w)^2)

    x = x + b*cos(theta)
    y = y + b*sin(theta)
    theta = theta + asin(sin(w) * f/d)
    v = v + h*a
    
    return [x, y, theta, v]
    
end

F (generic function with 6 methods)

In [16]:
ForwardDiff.jacobian(F, [1, 1, 3/2 * pi, 0, 1, 1])

4×6 Array{Float64,2}:
 1.0  0.0  0.0  -9.92519e-19  0.0  0.0 
 0.0  1.0  0.0  -0.00540302   0.0  0.0 
 0.0  0.0  1.0   0.00210368   0.0  0.0 
 0.0  0.0  0.0   1.0          0.0  0.01

In [1]:
using ForwardDiff

In [36]:
g = xu -> ForwardDiff.jacobian(F, xu); # g = ∇f

In [16]:
Array(1:4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [37]:
g(Array(1:6))

4×6 Array{Float64,2}:
 1.0  0.0  -0.00162717  -0.00289927   -0.0378654   0.0 
 0.0  1.0  -0.011415     0.000413281   0.00539758  0.0 
 0.0  0.0   1.0         -0.00239742    0.00283675  0.0 
 0.0  0.0   0.0          1.0           0.0         0.01

In [39]:
Base.IndexStyle(typeof(U))

IndexLinear()

In [35]:
a = supertype(typeof(U))

DenseArray{Array{Int64,1},1}

In [37]:
supertype(a)

AbstractArray{Array{Int64,1},1}

In [34]:
ReverseDiff.gradient(aux, [[i,i] for i in 1:10])

MethodError: MethodError: no method matching zero(::Type{Array{Int64,1}})
Closest candidates are:
  zero(!Matched::Type{LibGit2.GitHash}) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LibGit2\src\oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Pkg\src\resolve\VersionWeights.jl:19
  zero(!Matched::Type{Pkg.Resolve.MaxSum.FieldValues.FieldValue}) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\Pkg\src\resolve\FieldValues.jl:44
  ...

In [17]:
z(x, p) = sqrt(x^2 + p^2) - p

z (generic function with 1 method)

In [18]:
Lf(x, p, weight = 100000) =
    weight * (z(x[1], p[1]) + z(x[2], p[2]) + z(x[3], p[3]) + z(x[4], p[4]))

Lf (generic function with 2 methods)

In [19]:
# L(x, p = [0.1, 0.1, 0.01, 1], c_w = 0.01, c_a = 0.0001) =
#     0.01*(z(x[1], p[1]) + z(x[2], p[2])) + c_w * x[3]^2 + c_a * x[4]^2

L (generic function with 3 methods)

In [10]:
function L(x_u, p = [0.1, 0.1, 0.01, 1] , c_w = 0.01, c_a = 0.0001)
    x, y, theta, v, w, a = x_u
    p_x, p_y, p_theta, p_v = p
    0.01(z(x, p_x) + z(y, p_y)) + c_w * w^2 + c_a * a^2
end

L (generic function with 4 methods)

In [40]:
function J(U::Vector, x0 = [1, 1, 3/2 * pi, 0], N = 10)#total loss
    X = [x0]
    for j in 1:N-1
        push!(X, F(X[j], U[2j-1:2j]))
    end
    return sum([L(X[i], U[2i-1:2i]) for i in 1:N-1]) #+ Lf(X[N])
end

J (generic function with 3 methods)

In [11]:
function j(U, x0 = [1, 1, 3/2 * pi, 0], N = 2)#total loss
    X = [x0]
#     for j in 1:N-1
        push!(X, F(X[1], U[1:2]))
#     end
#     return sum([L(X[i], U[2i-1:2i]) for i in 1:N-1]) #+ Lf(X[N])
    L(X[2], U)
    
end

j (generic function with 3 methods)

In [12]:
ReverseDiff.gradient(j, [i for i in 1:2])

InexactError: InexactError: Int64(Int64, 0.5403023058681398)

In [30]:
L(1:6)

0.28167486001562164

In [86]:
ForwardDiff.gradient(L, 1:4)

4-element Array{Float64,1}:
 0.009950371902099893
 0.009987523388778447
 0.06                
 0.0008              

In [68]:
j=1

1

In [69]:
X=2

2

In [70]:
Xj

UndefVarError: UndefVarError: Xj not defined

In [9]:
function f(U::Vector, x0 = [1, 1, 3/2 * pi, 0], N = 5) 
#     x = x0
#     for j in 1:N
    x = F(x0, U)
#     end
    x
end

f (generic function with 3 methods)

In [3]:
f(u) = F([1, 1, 3/2 * pi, 0], u)

f (generic function with 1 method)

In [15]:
f([1,2])

4-element Array{Float64,1}:
 1.0             
 1.0             
 4.71238898038469
 0.02            

In [44]:
F(vcat(x0, [1,1]))

4-element Array{Float64,1}:
 1.0             
 1.0             
 4.71238898038469
 0.01            

In [48]:
f([i for i in 1:20])

4-element Array{Float64,1}:
 0.9999997675441916 
 1.000945174918594  
 4.712655352968368  
 0.30000000000000004

In [2]:
using ForwardDiff

In [4]:
j = U -> ForwardDiff.gradient(f, U)

#3 (generic function with 1 method)

In [5]:
j([i for i in 1:2])

MethodError: MethodError: no method matching extract_gradient!(::Type{ForwardDiff.Tag{typeof(f),Int64}}, ::Array{Array{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f),Int64},Float64,2},1},1}, ::Array{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f),Int64},Float64,2},1})
Closest candidates are:
  extract_gradient!(::Type{T}, ::AbstractArray, !Matched::ForwardDiff.Dual) where T at C:\Users\limen\.julia\packages\ForwardDiff\N0wMF\src\gradient.jl:77
  extract_gradient!(::Type{T}, ::AbstractArray, !Matched::Real) where T at C:\Users\limen\.julia\packages\ForwardDiff\N0wMF\src\gradient.jl:76
  extract_gradient!(::Type{T}, !Matched::DiffResults.DiffResult, !Matched::ForwardDiff.Dual) where T at C:\Users\limen\.julia\packages\ForwardDiff\N0wMF\src\gradient.jl:71
  ...

In [20]:
function J(U::Vector, x0 = [1, 1, 3/2 * pi, 0], N = 10)#total loss
    X = [x0 for i in 1:10]
    for j in 1:N-1
        push!(X, F(X[j], U[2j-1:2j+1]))
    end
    return sum([L(X[i], U[2i-1:2i]) for i in 1:N-1]) #+ Lf(X[N])
end

J (generic function with 3 methods)

In [22]:
ReverseDiff.gradient(J, [i for i in 1:20])

MethodError: MethodError: no method matching J(::ReverseDiff.TrackedArray{Int64,Int64,1,Array{Int64,1},Array{Int64,1}})
Closest candidates are:
  J(!Matched::Array{T,1} where T) at In[20]:2
  J(!Matched::Array{T,1} where T, !Matched::Any) at In[20]:2
  J(!Matched::Array{T,1} where T, !Matched::Any, !Matched::Any) at In[20]:2

In [34]:
U = [i for i in 1:20]

20-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20

In [24]:
x0 = [1, 1, 3/2 * pi, 0]

4-element Array{Float64,1}:
 1.0             
 1.0             
 4.71238898038469
 0.0             

In [26]:
X = [x0 for i in 1:10]

10-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]
 [1.0, 1.0, 4.71239, 0.0]

In [27]:
X[1]

4-element Array{Float64,1}:
 1.0             
 1.0             
 4.71238898038469
 0.0             

In [23]:
vcat([1, 1, 3/2 * pi, 0], U[1:2])

6-element Array{Float64,1}:
 1.0             
 1.0             
 4.71238898038469
 0.0             
 1.0             
 2.0             

In [32]:
j =  U -> ForwardDiff.gradient(J, U);

In [39]:
j(Array([i for i in 1:18]))

MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{typeof(J),Int64},Float64,9})
Closest candidates are:
  Float64(::Real, !Matched::RoundingMode) where T<:AbstractFloat at rounding.jl:185
  Float64(::T<:Number) where T<:Number at boot.jl:725
  Float64(!Matched::Int8) at float.jl:60
  ...

In [54]:
foo(U) = sum(i for u in U for i in u)

foo (generic function with 1 method)

In [51]:
foo([[i,i] for i in 1:10])

110

In [52]:
j =  U -> ForwardDiff.jacobian(foo, U);

In [38]:
for i in 1:3-1
    println(i)
end

1
2


In [39]:
Array(1:3)

3-element Array{Int64,1}:
 1
 2
 3

In [41]:
[1 2 3]

1×3 Array{Int64,2}:
 1  2  3

In [42]:
pi

π = 3.1415926535897...

In [11]:
f(3)

4

In [10]:
function f(x)
    y = x+1
    y
end

f (generic function with 1 method)

In [2]:
x = DDP(1,2,[1],[1,1])

ErrorException: x0 and p must have the same size

In [5]:
size([1,1]) 

(2,)

In [3]:
size([1]) == size([2,2])

false

In [15]:
f(x,y ) = 
size(x) == size(y) ?
1 : 2

f (generic function with 1 method)

In [16]:
f(1,[2])

2